# Vaults - Setup

## Implementation Notes
- Call totalAssets, totalCollectableFees, and totalSupply from 0x221B2D9aD7B994861Af3f4c8A80c86C4aa86Bf53 via web3.py
- Get transfers from subgraph

In [2]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
from mezo.currency_utils import get_token_price, format_musd_currency_columns
from mezo.datetime_utils import format_datetimes
from mezo.data_utils import add_rolling_values, add_pct_change_columns, add_cumulative_columns
from mezo.clients import BigQueryClient, SubgraphClient, Web3Client

load_dotenv(dotenv_path='../.env', override=True)
COINGECKO_KEY = os.getenv('COINGECKO_KEY')

In [3]:
# Subgraph Setup

AUGUST_VAULT_SUBGRAPH = "https://api.goldsky.com/api/public/project_cm6ks2x8um4aj01uj8nwg1f6r/subgraphs/mezo-vaults-mezo/1.0.0/gn"

SUBGRAPH_HEADERS = {
        "Content-Type": "application/json",
    }

GET_VAULT_TRANSFERS = """
query vaultTransfers($skip: Int!) {
  transfers(
    first: 1000
    orderBy: timestamp_
    orderDirection: desc
    skip: $skip
) {
    timestamp_
    from
    to
    value
    transactionHash_
    block_number
  }
}
"""

In [4]:
# Instantiate subgraph

vault = SubgraphClient(
    url = AUGUST_VAULT_SUBGRAPH, 
    headers = SUBGRAPH_HEADERS
)

In [5]:
# Fetch subgraph data
vaults =  vault.fetch_subgraph_data(GET_VAULT_TRANSFERS, 'transfers')

Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records found.


In [6]:
# Save vaults data df

vaults_df = pd.DataFrame(vaults)
display(vaults_df)

,timestamp_,from,to,value,transactionHash_,block_number
0,1752784316,0x0000000000000000000000000000000000000000,0x5a38f76b4a64873957dbcc4f72b41cd6b6491958,10000000000000000000,0x510ce30e718580509f62af058ada3751ace11c2bf893...,1768804
1,1752783862,0x0000000000000000000000000000000000000000,0xaded2abf6f2d3a1f3b7666cc684ef4ad4fc38769,1187501754112768164,0x492793d05745a2d8a44543adf7b680edbb5cd4b53a6f...,1768678
2,1752780797,0x0000000000000000000000000000000000000000,0xf3101c6a4ebd0d76518f3bd513d7a66a2e033454,10000000000000000000000,0xa800cff4977781132b769961210bfa215fbc2d61afb5...,1767826
3,1752775706,0x0000000000000000000000000000000000000000,0x34d3d2ef5967161f6766681d1106729bc19311db,5000000000000000000,0x5c9ea2473b465f123ddd4ff6733c8ec2ae88ed006c89...,1766410
4,1752766051,0x0000000000000000000000000000000000000000,0x69f592d7272bb4246155d8b5046f57da4a716135,20000000000000000000,0xb7c0fe32f8ad2d07e7787284f0a2da5695250eaab215...,1763721
...,...,...,...,...,...,...
71,1750924920,0x0000000000000000000000000000000000000000,0x6e80164ea60673d64d5d6228beb684a1274bb017,1000000000000000000,0xf94447d87f136b744db7d595d441d468fca44162c337...,1257108
72,1750899066,0x0000000000000000000000000000000000000000,0x83db892a6688b966122be219fa873bf24d7db79f,5000000000000000000,0x6a633ddd97ece5fd9c72271c72745a20c75635f93932...,1250310
73,1750894224,0x0000000000000000000000000000000000000000,0x83db892a6688b966122be219fa873bf24d7db79f,10000000000000000000,0xb1414979063a26c67326912e08202b38a096e43456aa...,1249036
74,1750853682,0x0000000000000000000000000000000000000000,0x6e80164ea60673d64d5d6228beb684a1274bb017,1000000000000000000,0xeadd15fb5deaf6f11e2bfd1a9421ae40de826f8a5e3e...,1238184


In [7]:
# Exploration 
format_musd_currency_columns(vaults_df, ['value'])

,timestamp_,from,to,value,transactionHash_,block_number
0,1752784316,0x0000000000000000000000000000000000000000,0x5a38f76b4a64873957dbcc4f72b41cd6b6491958,10.000000,0x510ce30e718580509f62af058ada3751ace11c2bf893...,1768804
1,1752783862,0x0000000000000000000000000000000000000000,0xaded2abf6f2d3a1f3b7666cc684ef4ad4fc38769,1.187502,0x492793d05745a2d8a44543adf7b680edbb5cd4b53a6f...,1768678
2,1752780797,0x0000000000000000000000000000000000000000,0xf3101c6a4ebd0d76518f3bd513d7a66a2e033454,10000.000000,0xa800cff4977781132b769961210bfa215fbc2d61afb5...,1767826
3,1752775706,0x0000000000000000000000000000000000000000,0x34d3d2ef5967161f6766681d1106729bc19311db,5.000000,0x5c9ea2473b465f123ddd4ff6733c8ec2ae88ed006c89...,1766410
4,1752766051,0x0000000000000000000000000000000000000000,0x69f592d7272bb4246155d8b5046f57da4a716135,20.000000,0xb7c0fe32f8ad2d07e7787284f0a2da5695250eaab215...,1763721
...,...,...,...,...,...,...
71,1750924920,0x0000000000000000000000000000000000000000,0x6e80164ea60673d64d5d6228beb684a1274bb017,1.000000,0xf94447d87f136b744db7d595d441d468fca44162c337...,1257108
72,1750899066,0x0000000000000000000000000000000000000000,0x83db892a6688b966122be219fa873bf24d7db79f,5.000000,0x6a633ddd97ece5fd9c72271c72745a20c75635f93932...,1250310
73,1750894224,0x0000000000000000000000000000000000000000,0x83db892a6688b966122be219fa873bf24d7db79f,10.000000,0xb1414979063a26c67326912e08202b38a096e43456aa...,1249036
74,1750853682,0x0000000000000000000000000000000000000000,0x6e80164ea60673d64d5d6228beb684a1274bb017,1.000000,0xeadd15fb5deaf6f11e2bfd1a9421ae40de826f8a5e3e...,1238184


In [8]:
format_datetimes(vaults_df, ['timestamp_'])

,timestamp_,from,to,value,transactionHash_,block_number
0,2025-07-17,0x0000000000000000000000000000000000000000,0x5a38f76b4a64873957dbcc4f72b41cd6b6491958,10.000000,0x510ce30e718580509f62af058ada3751ace11c2bf893...,1768804
1,2025-07-17,0x0000000000000000000000000000000000000000,0xaded2abf6f2d3a1f3b7666cc684ef4ad4fc38769,1.187502,0x492793d05745a2d8a44543adf7b680edbb5cd4b53a6f...,1768678
2,2025-07-17,0x0000000000000000000000000000000000000000,0xf3101c6a4ebd0d76518f3bd513d7a66a2e033454,10000.000000,0xa800cff4977781132b769961210bfa215fbc2d61afb5...,1767826
3,2025-07-17,0x0000000000000000000000000000000000000000,0x34d3d2ef5967161f6766681d1106729bc19311db,5.000000,0x5c9ea2473b465f123ddd4ff6733c8ec2ae88ed006c89...,1766410
4,2025-07-17,0x0000000000000000000000000000000000000000,0x69f592d7272bb4246155d8b5046f57da4a716135,20.000000,0xb7c0fe32f8ad2d07e7787284f0a2da5695250eaab215...,1763721
...,...,...,...,...,...,...
71,2025-06-26,0x0000000000000000000000000000000000000000,0x6e80164ea60673d64d5d6228beb684a1274bb017,1.000000,0xf94447d87f136b744db7d595d441d468fca44162c337...,1257108
72,2025-06-26,0x0000000000000000000000000000000000000000,0x83db892a6688b966122be219fa873bf24d7db79f,5.000000,0x6a633ddd97ece5fd9c72271c72745a20c75635f93932...,1250310
73,2025-06-25,0x0000000000000000000000000000000000000000,0x83db892a6688b966122be219fa873bf24d7db79f,10.000000,0xb1414979063a26c67326912e08202b38a096e43456aa...,1249036
74,2025-06-25,0x0000000000000000000000000000000000000000,0x6e80164ea60673d64d5d6228beb684a1274bb017,1.000000,0xeadd15fb5deaf6f11e2bfd1a9421ae40de826f8a5e3e...,1238184


In [9]:
tvl = sum(vaults_df['value'])

In [25]:
depositors = pd.DataFrame(vaults_df['to'].unique()).count()
deposits = vaults_df['to'].count()

In [10]:
vaults_df.dtypes

timestamp_           object
from                 object
to                   object
value               float64
transactionHash_     object
block_number         object
dtype: object

In [11]:
vaults_df['count'] = 1

In [17]:
daily_vault_txns = vaults_df.groupby(['timestamp_']).agg(
    value = ('value', 'sum'),
    transactions = ('count', 'sum'),
    users = ('to', lambda x: x.nunique())
).reset_index().sort_values(by='timestamp_', ascending=False)

In [26]:
daily_vault_txns

,timestamp_,value,transactions,users
4,2025-07-17,31456.294951,21,21
3,2025-07-16,87524.961135,42,32
2,2025-07-14,1.046000,1,1
1,2025-06-26,10.100223,9,4
0,2025-06-25,12.000000,3,2


In [ ]:
# instantiate web3 client

w3 = Web3Client('AugustVault')
august_vault = w3.load_contract()
august_vault.all_functions()

In [ ]:
# instantiate bigquery
bq = BigQueryClient(key='GOOGLE_CLOUD_KEY', project_id='mezo-portal-data')

In [ ]:
GET_POOL_FEES = """
query poolFees ($skip: Int!) {
  fees(
    orderBy: timestamp_
    orderDirection: desc
    first: 1000
    skip: $skip
) {
    timestamp_
    sender
    amount0
    amount1
    contractId_
    transactionHash_
  }
}
"""